In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lbg
from fastai.structured import *
from fastai.column_data import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.model_selection import StratifiedKFold
fk = StratifiedKFold(n_splits=10)
from sklearn.metrics import accuracy_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import IsolationForest
import os
print(os.listdir("../input"))
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('../input/train_LZdllcl.csv')
test = pd.read_csv('../input/test_2umaH9m.csv')

In [ ]:
train.shape,test.shape

In [ ]:
train.head(1)

In [ ]:
print(train.isnull().sum())

In [ ]:
#train = train[(train.no_of_trainings)<4]

In [ ]:
train.describe()

In [ ]:
train['education'].fillna('none',inplace=True)
#train['previous_year_rating'].fillna(train['previous_year_rating'].mean(),inplace=True)
train['previous_year_rating'].fillna(0,inplace=True)
test['education'].fillna('none',inplace=True)
#test['previous_year_rating'].fillna(train['previous_year_rating'].mean(),inplace=True)
test['previous_year_rating'].fillna(0,inplace=True)
train['score'] = train['avg_training_score']/train['no_of_trainings']
test['score'] = test['avg_training_score']/test['no_of_trainings']

In [ ]:
col = ['department','region','education','gender','recruitment_channel',]
comb = train.append(test,sort = False)
for i in col:
    comb[i] = le.fit_transform(comb[i].astype('str'))
    comb[i] = comb[i].astype('int')
train = comb[:train.shape[0]]
test = comb[train.shape[0]:]

In [ ]:
cat_vars = [
            'department',
            'region',
            'education',
            'gender',
            'recruitment_channel',
            'KPIs_met >80%',
            'awards_won?',]
contin_vars = [
                'no_of_trainings',
                'age',
                'previous_year_rating',
                'length_of_service',
                'avg_training_score',
                'score']
objective = ['is_promoted']
ids = ['employee_id']

In [ ]:
for v in cat_vars:
    if v not in ['KPIs_met>80%','awards_won?']:
        train[v] = train[v].astype('category')
    else:
        train[v] = train[v].astype('category').cat.as_ordered()

In [ ]:
apply_cats(test,train)

In [ ]:
for v in contin_vars:
    train[v] = train[v].astype('float32')
    test[v] = test[v].astype('float32')

In [ ]:
#train = train.set_index('employee_id')
train = train[ids+cat_vars+contin_vars+objective]

In [ ]:
train.head(2)

In [ ]:
test.head(2)

In [ ]:
test = test.set_index('employee_id')
test['is_promoted'] = 0
test = test[cat_vars+contin_vars+['is_promoted']]
test_X = test.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12]]
test_y = test.iloc[:,[13]].values.astype('int32')

In [ ]:
test.head(2)

In [ ]:
train = train.set_index('employee_id')

In [ ]:
trainData, validData = train_test_split(train,
                                        test_size=0.3,
                                        stratify=train.is_promoted)

In [ ]:
trainData.head(1)

In [ ]:
train_X = trainData.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12]]
train_y = trainData.iloc[:,[13]].values.astype('int32')
#train_X, train_y, nas, mapper = proc_df(trainData, 'is_promoted')
#valid_X, valid_y, nas, mapper = proc_df(validData, 'is_promoted')
valid_X = validData.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12]]
valid_y = validData.iloc[:,[13]].values.astype('int32')

In [ ]:
GV = False
if GV:
    params = {'boosting_type': 'gbdt',
              'max_depth' : -1,
              'objective': 'binary',
              'nthread': 3,
              'num_leaves': 64,
              'learning_rate': 0.05,
              'max_bin': 512,
              'subsample_for_bin': 200,
              'subsample': 1,
              'subsample_freq': 1,
              'colsample_bytree': 0.8,
              'reg_alpha': 5,
              'reg_lambda': 10,
              'min_split_gain': 0.5,
              'min_child_weight': 1,
              'min_child_samples': 5,
              'scale_pos_weight': 1,
              'num_class' : 1,
              'metric' : 'auc'}
    gridParams = {
        'max_depth' : [15],
        'learning_rate': [0.005],
        'n_estimators': [2000],
        'num_leaves': [6,8,12,16,123],
        'boosting_type' : ['gbdt'],
        'objective' : ['binary'],
        'colsample_bytree' : [0.65],
        'subsample' : [0.7],
        'reg_alpha' : [1],
        'reg_lambda' : [1],
        }
    mdl = lbg.LGBMClassifier(boosting_type= 'gbdt',
              objective = 'binary',
              n_jobs = 3, # Updated from 'nthread'
              silent = True,
              max_depth = params['max_depth'],
              max_bin = params['max_bin'],
              subsample_for_bin = params['subsample_for_bin'],
              subsample = params['subsample'],
              subsample_freq = params['subsample_freq'],
              min_split_gain = params['min_split_gain'],
              min_child_weight = params['min_child_weight'],
              min_child_samples = params['min_child_samples'],
              scale_pos_weight = params['scale_pos_weight'])

    grid = GridSearchCV(mdl, gridParams,
                        verbose=0,
                        cv=2,
                        n_jobs=2)

    grid.fit(train_X,train_y)

    print(grid.best_params_)
    print(grid.best_score_)

    # Using parameters already set above, replace in the best from the grid search
    params['colsample_bytree'] = grid.best_params_['colsample_bytree']
    params['learning_rate'] = grid.best_params_['learning_rate']
    params['num_leaves'] = grid.best_params_['num_leaves']
    params['reg_alpha'] = grid.best_params_['reg_alpha']
    params['reg_lambda'] = grid.best_params_['reg_lambda']
    params['subsample'] = grid.best_params_['subsample']

    print(params)

In [ ]:
train_y = train_y.astype(np.int8)
train_y[train_y==1] = -1
train_y[train_y==0] = 1

valid_y = valid_y.astype(np.int8)
valid_y[valid_y==1] = -1
valid_y[valid_y==0] = 1

In [ ]:
skf = StratifiedKFold(n_splits=3) 
folds = list(skf.split(train_X, train_y))
forest = IsolationForest()
forest.contamination = sum(train_y==-1)/len(train_y)

In [ ]:
ps = {"n_estimators": (55, 75, 95, 115)}
f1sc = make_scorer(f1_score)
search = GridSearchCV(estimator=forest, param_grid=ps, scoring=f1sc, cv=folds,)
search.fit(train_X, train_y)

In [ ]:
"""lgbm_model = lbg.LGBMClassifier(boosting_type = 'gbdt',
                                n_estimators=2000,
                                max_depth = 15,
                                objective = 'binary',
                                num_leaves = 64,
                                learning_rate = 0.005,
                                max_bin = 512,
                                subsample_for_bin = 200,
                                subsample = 0.7,
                                subsample_freq = 1,
                                colsample_bytree = 0.65,
                                reg_alpha = 1,
                                reg_lambda = 1,
                                min_split_gain = 0.01,
                                min_child_weight = 1,
                                min_child_samples = 5,
                                scale_pos_weight = 1,
                                metric = 'binary_logloss')
lgbm_model.fit(train_X,train_y)
pred = lgbm_model.predict(valid_X)"""

In [ ]:
df = pd.DataFrame(search.cv_results_)
df[["mean_test_score", "std_test_score", "params"]].sort_values(by=["mean_test_score"], ascending=False).head()

In [ ]:
test_X.head(2)

In [ ]:
optimal_forest = search.best_estimator_
pred = optimal_forest.predict(valid_X)

In [ ]:
f1_score(valid_y,pred)

In [ ]:
"""
lgbm_model = lbg.LGBMClassifier(
            n_estimators=2000,
            objective = 'binary',
            boosting_type = 'gbdt',
            learning_rate = 0.1,
            min_split_gain =  0.01,
            num_leaves = 123,
            metric = 'binary_logloss')
lgbm_model.fit(train_X,train_y)
pred = lgbm_model.predict(test_X)
"""

In [ ]:
sub_ = True
if sub_:
    pred = optimal_forest.predict(test_X)
    sub = pd.read_csv('../input/sample_submission_M0L0uXE.csv')
    sub['is_promoted'] = pred
    sub[sub==1] = 0
    sub[sub==-1] = 1
    sub.head()

    sub.hist('is_promoted')
    sub.to_csv("submission_fn_1.csv",index=False)